This is Luke's 1.0 d4d work.

All you would need to do to run this is update 
 os.chdir('C:\\Users\\...\\geojsons') 
 and
 curr_target_dir_path = 'C:\\Users\\...\\geojsons\\' + curr_target_dir
 in the 4th and 5th cell (this cell being the 1st cell)

As of now this program: 
Enters folder in 'geojsons', loads arrival and departure files, pulls some data from each file,  exits folder, repeats. 
 
Next step is to allow user to easily filter such as the following coordinate box provided by connor:
1 - (33.272014, 42.447810), 2 - (33.068781, 41.775133), 3 - ( 32.116345, 42.469610), 4 - (32.394570, 43.127131)

The following code is my first set of work, but I realized that I can be doing this in a much more efficient way, which will be the 2.0 document. Thanks.

In [1]:
import pandas as pd
import numpy as np 
import geopandas as gpd

In [2]:
import scipy.stats as stats
import math
import matplotlib.pylab as plt
%matplotlib inline

In [3]:
import os

os.chdir('C:\\Users\\luker\\Desktop\\d4d_work\\geojsons')


In [4]:
arrival_date_list = [] # track all dates of arrivals
departure_date_list = [] # track all dates of departures


dir_list = os.listdir('.') 
print("Number of folders containing geojson arrival/departure file pairs: " + str(len(dir_list)))
#for i in range(len(dir_list)):
#    curr_target_dir = dir_list[i]

for i in range(5):
    #initialize arrival tuple attributes to zero
    arrival_count = 0
    arrival_avg_weighted_lat = 0
    arrival_avg_weighted_lon = 0
    arrival_avg_change_area = 0
    arrival_avg_pixel_count = 0
    arrival_avg_perimeter = 0
    arrival_avg_target_mean_return = 0
    
    
    #initialize departure tuple attributes to zero
    departure_count = 0
    departure_avg_weighted_lat = 0
    departure_avg_weighted_lon = 0
    departure_avg_change_area = 0
    departure_avg_pixel_count = 0
    departure_avg_perimeter = 0
    departure_avg_target_mean_return = 0
    
    #file setup
    curr_target_dir = dir_list[i]
    curr_target_dir_path = 'C:\\Users\\luker\\Desktop\\d4d_work\\geojsons\\' + curr_target_dir
    os.chdir(curr_target_dir_path)
    file_list = os.listdir('.') 
    arrivalfname = r""+file_list[0] #arrival is always first (but also checked with if statement in processing code)
    dfArrival = gpd.read_file(arrivalfname)
    departurefname = r""+file_list[1]      
    dfDeparture = gpd.read_file(departurefname)
    
    #arrival file processing
    if dfArrival['detect_type'][0] == 'Arrival':
        print("Processing arrival: " + str(i))
        arrival_date = dfArrival['timestamp'][0]
        arrival_date = arrival_date[0:10]
        #check if arrival_date in arrival_date_list 
        if arrival_date not in arrival_date_list:
            arrival_date_list.append(arrival_date)
        arrival_count = dfArrival['geometry'].count()
        #initialize low/high lat/lon to first point
        arrival_lowest_weighted_lat = dfArrival['weighted_centroid_lat'][0]
        arrival_highest_weighted_lat = dfArrival['weighted_centroid_lat'][0]
        arrival_lowest_weighted_lon = dfArrival['weighted_centroid_lon'][0]
        arrival_highest_weighted_lon = dfArrival['weighted_centroid_lon'][0]
        
        
        for j in range(arrival_count):
            #set current arrival iterations weighted lat/lon 
            curr_arr_weight_lat = dfArrival['weighted_centroid_lat'][j]
            curr_arr_weight_lon = dfArrival['weighted_centroid_lon'][j]
            if curr_arr_weight_lat < arrival_lowest_weighted_lat:
                arrival_lowest_weighted_lat = curr_arr_weight_lat
            if curr_arr_weight_lat > arrival_highest_weighted_lat:
                arrival_highest_weighted_lat = curr_arr_weight_lat
            if curr_arr_weight_lon < arrival_lowest_weighted_lon:
                arrival_lowest_weighted_lon = curr_arr_weight_lon
            if curr_arr_weight_lon > arrival_highest_weighted_lon:
                arrival_highest_weighted_lon = curr_arr_weight_lon
            arrival_avg_weighted_lat += curr_arr_weight_lat
            arrival_avg_weighted_lon += curr_arr_weight_lon
            #Was about to implement MIN/MAX when I realized I am not properly leveraging data science concepts.
            curr_arr_change_area = dfArrival['change_area'][j]
            arrival_avg_change_area += curr_arr_change_area
            curr_arr_pixel_count = dfArrival['pixel_count'][j]
            arrival_avg_pixel_count += curr_arr_pixel_count
            curr_arr_perimeter = dfArrival['perimeter'][j]
            arrival_avg_perimeter += curr_arr_perimeter
            curr_arr_target_mean_return = dfArrival['target_mean_return'][j]
            arrival_avg_target_mean_return += curr_arr_target_mean_return

            
        #perform averaging calculations
        arrival_avg_weighted_lat = round( (arrival_avg_weighted_lat/arrival_count) ,6)
        arrival_avg_weighted_lon = round( (arrival_avg_weighted_lon/arrival_count) ,6)
        arrival_avg_change_area = round( (arrival_avg_change_area/arrival_count) ,2)
        arrival_avg_pixel_count = round( (arrival_avg_pixel_count/arrival_count) ,1)
        arrival_avg_perimeter = round( (arrival_avg_perimeter/arrival_count) ,2)
        arrival_avg_target_mean_return = round( (arrival_avg_target_mean_return/arrival_count) ,2)
        print("arrival_date: " + str(arrival_date))
        print("arrival_count: " + str(arrival_count))
        print("arrival_avg_change_area: " + str(arrival_avg_change_area))
        print("arrival_avg_pixel_count: " + str(arrival_avg_pixel_count))
        print("arrival_avg_perimeter: " + str(arrival_avg_perimeter))
        print("arrival_avg_target_mean_return: " + str(arrival_avg_target_mean_return))
        print("arrival_avg_weighted_lat: " + str(arrival_avg_weighted_lat))
        print("arrival_avg_weighted_lon: " + str(arrival_avg_weighted_lon))
        print("arrival_lowest_weighted_lat: " + str(arrival_lowest_weighted_lat))
        print("arrival_highest_weighted_lat: " + str(arrival_highest_weighted_lat))
        print("arrival_lowest_weighted_lon: " + str(arrival_lowest_weighted_lon))
        print("arrival_highest_weighted_lon: " + str(arrival_highest_weighted_lon))
        print("")        
                 
            
    #departure file processing
    if dfDeparture['detect_type'][0] == 'Departure':
        print("Processing departure: " + str(i))
        departure_date = dfDeparture['timestamp'][0]
        departure_date = departure_date[0:10]
        #check if departure_date in departure_date_list 
        if departure_date not in departure_date_list:
            departure_date_list.append(departure_date)
        departure_count = dfDeparture['geometry'].count()
        #initialize low/high lat/lon to first point
        departure_lowest_weighted_lat = dfDeparture['weighted_centroid_lat'][0]
        departure_highest_weighted_lat = dfDeparture['weighted_centroid_lat'][0]
        departure_lowest_weighted_lon = dfDeparture['weighted_centroid_lon'][0]
        departure_highest_weighted_lon = dfDeparture['weighted_centroid_lon'][0]
        for k in range(departure_count):
            #set current iterations weighted lat/lon 
            curr_dep_weight_lat = dfDeparture['weighted_centroid_lat'][k]
            curr_dep_weight_lon = dfDeparture['weighted_centroid_lon'][k]
            if curr_dep_weight_lat < departure_lowest_weighted_lat:
                departure_lowest_weighted_lat = curr_dep_weight_lat
            if curr_dep_weight_lat > departure_highest_weighted_lat:
                departure_highest_weighted_lat = curr_dep_weight_lat
            if curr_dep_weight_lon < departure_lowest_weighted_lon:
                departure_lowest_weighted_lon = curr_dep_weight_lon
            if curr_dep_weight_lon > departure_highest_weighted_lon:
                departure_highest_weighted_lon = curr_dep_weight_lon
            departure_avg_weighted_lat += curr_dep_weight_lat
            departure_avg_weighted_lon += curr_dep_weight_lon
            departure_avg_change_area += dfDeparture['change_area'][k]
            departure_avg_pixel_count += dfDeparture['pixel_count'][k]
            departure_avg_perimeter += dfDeparture['perimeter'][k]
            departure_avg_target_mean_return += dfDeparture['target_mean_return'][k]
            
        departure_avg_weighted_lat = round( (departure_avg_weighted_lat/departure_count) ,6)
        departure_avg_weighted_lon = round( (departure_avg_weighted_lon/departure_count) ,6)
        departure_avg_change_area = round( (departure_avg_change_area/departure_count) ,2)
        departure_avg_pixel_count = round( (departure_avg_pixel_count/departure_count) ,1)
        departure_avg_perimeter = round( (departure_avg_perimeter/departure_count) ,2)
        departure_avg_target_mean_return = round( (departure_avg_target_mean_return/departure_count) ,2)
        print("departure_date: " + str(departure_date))
        print("departure_count: " + str(departure_count))
        print("departure_avg_weighted_lat: " + str(departure_avg_weighted_lat))
        print("departure_avg_weighted_lon: " + str(departure_avg_weighted_lon))
        print("departure_avg_change_area: " + str(departure_avg_change_area))
        print("departure_avg_pixel_count: " + str(departure_avg_pixel_count))
        print("departure_avg_perimeter: " + str(departure_avg_perimeter))
        print("departure_avg_target_mean_return: " + str(departure_avg_target_mean_return))
        print("departure_lowest_weighted_lat: " + str(departure_lowest_weighted_lat))
        print("departure_highest_weighted_lat: " + str(departure_highest_weighted_lat))
        print("departure_lowest_weighted_lon: " + str(departure_lowest_weighted_lon))
        print("departure_highest_weighted_lon: " + str(departure_highest_weighted_lon))
        print("")
        print("")
        
    #arrival_date_count_list.append(arrival_tuple)
    #departure_date_count_list.append(departure_tuple) 

print("")
print("")
print("")
print("arrival_date_list: ")
print(arrival_date_list)
print("departure_date_list: ")
print(departure_date_list)




Number of folders containing geojson arrival/departure file pairs: 1159
Processing arrival: 0
arrival_date: 2018-11-13
arrival_count: 5237
arrival_avg_change_area: 606.28
arrival_avg_pixel_count: 10.8
arrival_avg_perimeter: 122.92
arrival_avg_target_mean_return: 242.13
arrival_avg_weighted_lat: 36.945544
arrival_avg_weighted_lon: 45.676988
arrival_lowest_weighted_lat: 36.063761
arrival_highest_weighted_lat: 37.777945
arrival_lowest_weighted_lon: 43.851022
arrival_highest_weighted_lon: 46.929198

Processing departure: 0
departure_date: 2018-11-01
departure_count: 2910
departure_avg_weighted_lat: 36.858341
departure_avg_weighted_lon: 45.111889
departure_avg_change_area: 454.66
departure_avg_pixel_count: 8.8
departure_avg_perimeter: 114.84
departure_avg_target_mean_return: 236.59
departure_lowest_weighted_lat: 36.083356
departure_highest_weighted_lat: 37.768096
departure_lowest_weighted_lon: 43.858067
departure_highest_weighted_lon: 46.842434


Processing arrival: 1
arrival_date: 2018-11-

The cell below is the reason I am no longer wasting time with actually implementations by hand. Should've noticed sooner, but live and learn!

In [5]:
dfArrival.describe()

,shape_orientation,change_area,weighted_centroid_lat,target_minimum_return,background_minimum_return,ref_graze_angle,target_maximum_return,sec_azimuth_angle,ref_azimuth_angle,shape_extent_minor,...,pixel_count,centroid_lat,target_total_return,detect_strength,eccentricity,centroid_lon,target_mean_return,sec_graze_angle,background_mean_return,compactness
count,1620.000000,1620.000000,1620.000000,1620.000000,1620.000000,1620.000000,1620.000000,1620.000000,1620.000000,1620.000000,...,1620.000000,1620.000000,1.620000e+03,1620.000000,1620.000000,1620.000000,1620.000000,1620.000000,1620.000000,1620.000000
mean,114.433246,543.913191,29.588152,120.186549,0.666988,50.914879,3315.885654,259.635976,259.639646,24.062160,...,10.287037,29.588151,1.225567e+04,27.508395,0.834307,47.926851,917.985000,50.916086,3.415568,1.172219
std,33.820120,763.443991,0.495150,226.751760,0.902165,1.922843,11023.490323,0.171131,0.170913,9.586431,...,14.525383,0.495150,4.985718e+04,7.338542,0.125425,0.319690,1835.079496,1.922918,2.241432,0.257749
min,0.000000,282.650000,28.860946,0.000000,0.000000,46.060951,45.320000,259.192864,259.192483,10.140000,...,6.000000,28.860962,1.897800e+02,11.600000,0.000000,46.903813,23.720000,46.061932,0.130000,0.581800
25%,101.662450,315.440000,29.181387,53.797500,0.050000,50.685500,539.222500,259.637685,259.642248,19.410000,...,6.000000,29.181391,1.930730e+03,22.400000,0.779500,47.752010,260.097500,50.686712,2.017500,0.984200
50%,120.963800,368.100000,29.330000,81.715000,0.330000,50.689393,1055.085000,259.645220,259.647336,21.480000,...,7.000000,29.329961,3.593105e+03,27.000000,0.856700,47.909418,435.915000,50.690606,3.020000,1.084600
75%,135.000000,525.390000,30.054749,120.807500,0.962500,50.697532,2461.820000,259.667165,259.670293,26.090000,...,10.000000,30.054734,8.316943e+03,31.600000,0.923675,48.045873,888.452500,50.698814,4.310000,1.298400
max,177.618200,22396.420000,30.523965,6954.150000,6.710000,56.095530,184300.780000,260.112108,260.117187,143.430000,...,426.000000,30.523959,1.457850e+06,63.900000,0.997100,49.128789,30995.540000,56.096936,18.230000,2.171000
